In [1]:
#added to use driver
import csv
import numpy as np
from neo4j import GraphDatabase
import pandas as pd
import operator
import re
import math
import random
from random import randint
import numpy as np
import argparse
import torch
import torch.autograd as autograd
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from datetime import datetime
import networkx as nx
from node2vec import Node2Vec
from gensim.models import Word2Vec

C:\Users\meetn\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "a"))

In [10]:
#no need to run again as  I have exported embedidng and model 

def createembedding(file, query):
    print('Create the edge list')
    with driver.session() as session, open(file, "w") as edges_file:
        result = session.run(query)

        writer = csv.writer(edges_file, delimiter=" ")

        for row in result:
            writer.writerow([row["source"], row["target"]])

In [ ]:
def createNode2Vec(file, EMBEDDING_FILENAME, EMBEDDING_MODEL_FILENAME):   
    # Create a graph
    graph=nx.read_edgelist(file, create_using = nx.DiGraph(), nodetype = None, data = [('weight', int)])

    # Precompute probabilities and generate walks
    node2vec = Node2Vec(graph, dimensions=10, walk_length=3, num_walks=20, workers=4)

    ## if d_graph is big enough to fit in the memory, pass temp_folder which has enough disk space
    # Note: It will trigger "sharedmem" in Parallel, which will be slow on smaller graphs
    #node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4, temp_folder="/mnt/tmp_data")
    # Embed
    model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)

    # Look for most similar nodes
    #model.wv.most_similar('2')  # Output node names are always strings

    # Save embeddings for later use
    model.wv.save_word2vec_format(EMBEDDING_FILENAME)

    # Save model for later use
    model.save(EMBEDDING_MODEL_FILENAME)

In [11]:
#FILES - #no need to run again as  I have exported embedidng and model 
file = 'graph/movies.edgelist'
EMBEDDING_FILENAME = 'emb/node2vecmov.emb'
EMBEDDING_MODEL_FILENAME = 'emb/embdnode2vecmov.model'
movienode2vecquery = """\
        MATCH (m:Movie)--(other)
        RETURN id(m) AS source, id(other) AS target
        """

createembedding(file, movienode2vecquery)

Create the edge list


In [12]:
#no need to run again as  I have exported embedidng and model 
createNode2Vec(file, EMBEDDING_FILENAME, EMBEDDING_MODEL_FILENAME)

Computing transition probabilities: 100%|███████████████████████████████████████| 9980/9980 [00:00<00:00, 10788.50it/s]
C:\Users\meetn\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [97]:
#no need to run again as  I have exported embedidng and model 
file = 'graph/users.edgelist'
EMBEDDING_FILENAME = 'emb/node2vecuser.emb'
EMBEDDING_MODEL_FILENAME = 'emb/embdnode2vecuser.model'
usernode2vecquery = """\
        MATCH (u:User)--(other)
        RETURN id(u) AS source, id(other) AS target
        """
createembedding(file, usernode2vecquery)

Create the edge list


In [98]:
#no need to run again as  I have exported embedidng and model 
createNode2Vec(file, EMBEDDING_FILENAME, EMBEDDING_MODEL_FILENAME)

Computing transition probabilities: 100%|███████████████████████████████████████| 6257/6257 [00:00<00:00, 26249.72it/s]
C:\Users\meetn\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [15]:
EMBEDDING_MODEL_FILENAME = 'emb/embdnode2vecmov.model'
model = Word2Vec.load(EMBEDDING_MODEL_FILENAME)
#model.train([["hello", "world"]], total_examples=1, epochs=1)

In [16]:
model.wv.most_similar('2')

[('5697', 0.8905104398727417),
 ('556', 0.8781545162200928),
 ('6899', 0.8701952695846558),
 ('3646', 0.8629645109176636),
 ('9559', 0.8580236434936523),
 ('6889', 0.8575080633163452),
 ('9129', 0.8424545526504517),
 ('106', 0.8383544683456421),
 ('2252', 0.8276394009590149),
 ('1993', 0.8236179947853088)]

In [3]:
import pandas as pd

movies_ratings_query = """\
MATCH (u:User)-[r:RATED]->(m:Movie)
WITH distinct r.rating as ratings ORDER BY r.rating
WITH collect(ratings) AS ratings
MATCH (u:User)-[r:RATED]->(m:Movie)
WITH ratings, id(u) AS source, m.embedding AS embedding, id(m) AS movie, r.rating AS target, r.timestamp AS Time
RETURN source, embedding, movie, target, Time, 
[r in ratings | CASE WHEN r in target THEN 1 ELSE 0 END] AS ratings ORDER BY Time ASC
"""

with driver.session() as session:
    result = session.run(movies_ratings_query)
    df = pd.DataFrame([dict(row) for row in result])

In [4]:
#What does our DataFrame look like at the moment?
df.head()

,source,embedding,movie,target,Time,ratings
0,9891,"[-0.0058479453, 0.0019804768, 0.0040325085, -0...",3058,3.0,1000899120,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
1,9957,"[-0.007443971, 0.0009501985, -0.003198112, 0.0...",644,3.0,1001975811,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,9957,"[0.0016236857, -0.0013796845, -0.0007558407, -...",2078,5.0,1001975853,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,9957,"[-0.001516504, -0.0076829926, -0.0056096804, 0...",1213,2.0,1001975853,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,9957,"[0.004472015, 0.0024092905, 0.0056105647, 0.00...",1110,4.0,1001975853,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


In [5]:
df.to_csv('Node2Vec_TrainingData.csv')
#df = pd.read_csv('Node2Vec_TrainingData.csv')

In [6]:
train_index = int(len(df) * 0.9)
train_data = df[:train_index]
test_data = df[train_index:]

train_x = train_data.loc[:, "embedding"]
train_x = pd.DataFrame(np.array([np.array(item) for item in train_x.values]))
train_x.columns = [str(col) for col in train_x.columns.get_values()]
train_y = train_data.loc[:, 'ratings']


C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  import sys


In [7]:
# separate test data
test_x = test_data.loc[:, "embedding"]
test_x = pd.DataFrame(np.array([np.array(item) for item in test_x.values]))
test_x.columns = [str(col) for col in train_x.columns.get_values()]

test_y = test_data.loc[:, 'ratings']

C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  after removing the cwd from sys.path.


Building Tensorflow model

In [11]:
from tensorflow.python.feature_column import feature_column_lib
from tensorflow.python.training import training_util
from tensorflow.python.training.ftrl import FtrlOptimizer

import tensorflow as tf

create a simple multi label classifier because a movie can have more than one genre associated with it

In [12]:
def train_input_fn(features, labels, batch_size):
    labels = tf.constant(np.array([np.array(item)
                                   for item in labels.values]))
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    return dataset.shuffle(1000).repeat().batch(batch_size)


def eval_input_fn(features, labels, batch_size):
    labels = tf.constant(np.array([np.array(item)
                                   for item in labels.values]))

    features = dict(features)
    inputs = (features, labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    assert batch_size is not None, "batch_size must not be None"
    return dataset.batch(batch_size)

In [13]:
def model_fn(features, labels, mode, config):
    def train_op_fn(loss):
        opt = FtrlOptimizer(learning_rate=LEARNING_RATE)
        return opt.minimize(loss,
                            global_step=training_util.get_global_step())

    def logit_fn(features):
        return feature_column_lib.linear_model(
            features=features,
            feature_columns=feature_columns,
            units=head.logits_dimension)

    return head.create_estimator_spec(
        features=features,
        mode=mode,
        logits=logit_fn(features=features),
        labels=labels,
        train_op_fn=train_op_fn)

In [14]:
def predict_input_fn(df_data, num_epochs):
        return tf.estimator.inputs.pandas_input_fn(
            x=df_data,
            num_epochs=num_epochs,
           shuffle=False
        )

In [16]:

feature_columns = [tf.feature_column.numeric_column(key=key)
                   for key in train_x.keys()]

LEARNING_RATE = 0.3
loss_reduction = tf.losses.Reduction.SUM_OVER_BATCH_SIZE

head = tf.compat.v1.estimator.MultiLabelHead(10, weight_column=None, label_vocabulary=None,loss_reduction=loss_reduction)

classifier = tf.estimator.Estimator(model_fn=model_fn)

classifier.train(
    input_fn=lambda: train_input_fn(train_x, train_y, 100),
    steps=2000)

eval_result = classifier.evaluate(
    input_fn=lambda: eval_input_fn(test_x, test_y, 100))

print(eval_result)
print('\nTest set AUC: {auc:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\meetn\\AppData\\Local\\Temp\\tmpclz8gzqr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002B29350EE88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for 

In [17]:
ratings_query = """\
MATCH (u:User)-[r:RATED]->(m:Movie)
WITH distinct r.rating as ratings ORDER BY r.rating
RETURN collect(ratings) AS ratings
"""

with driver.session() as session:
    result = session.run(ratings_query)
    ratings = result.peek()["ratings"]

ratings

[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

For each user, get 5 movies and get Top 5 similar movies, which the user has not rated yet.

In [77]:
#This is taking lots of time, hence would use pandas to generate the nodes. 
'''movies_ratings_predict_query = """\
MATCH (u:User)-[r0:RATED]->(m:Movie)<-[:RATED]-(o:User) 
MATCH (o)-[r:RATED]->(m1:Movie) WHERE NOT EXISTS( (u)-[:RATED]->(m1) ) 
WITH id(u) AS source, m1.embedding AS embedding, id(m1) AS movie
RETURN source, embedding, movie, 0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] AS ratings
ORDER BY RAND()
LIMIT 100
""" 

with driver.session() as session:
    result = session.run(movies_ratings_predict_query)
    predict_df = pd.DataFrame([dict(row) for row in result])
    
predict_df.to_csv('node2Vec_forprediction.csv')'''

'movies_ratings_predict_query = """MATCH (u:User)-[r0:RATED]->(m:Movie)<-[:RATED]-(o:User) \nMATCH (o)-[r:RATED]->(m1:Movie) WHERE NOT EXISTS( (u)-[:RATED]->(m1) ) \nWITH id(u) AS source, m1.embedding AS embedding, id(m1) AS movie\nRETURN source, embedding, movie, 0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] AS ratings\nORDER BY RAND()\nLIMIT 100\n""" \n\nwith driver.session() as session:\n    result = session.run(movies_ratings_predict_query)\n    predict_df = pd.DataFrame([dict(row) for row in result])\n    \npredict_df.to_csv(\'node2Vec_forprediction.csv\')'

Hence decided to get all users. Loop through user dataframe and pass each user to the below query, get the user-movies data for which user has not rated the movie

In [18]:
ratings_query = """\
MATCH (u:User)-[r:RATED]->(m:Movie)
RETURN distinct id(u) As source
"""

with driver.session() as session:
    result = session.run(ratings_query)
    allusers = pd.DataFrame([dict(row) for row in result])

allusers.head()

,source
0,9810
1,9829
2,5150
3,9889
4,9841


In [ ]:
movies_ratings_predict_query = """\
MATCH (u:User)-[r0:RATED]->(m:Movie)<-[:RATED]-(o:User) 
MATCH (o)-[r:RATED]->(m1:Movie) WHERE NOT EXISTS( (u:User {id: {sourceId}})-[:RATED]->(m1) )
WITH id(u) AS source, m1.embedding AS embedding, id(m1) AS movie
RETURN source, embedding, movie, 0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] AS ratings
ORDER BY RAND()
LIMIT 100
""" 
new_predict_df = pd.DataFrame() 
for index, row in allusers.iterrows():
    print(row['source'])
    with driver.session() as session:
        result = session.run(movies_ratings_predict_query, sourceId=str(row['source']))
        predict_df = pd.DataFrame([dict(row) for row in result]) 
        new_predict_df.append(predict_df)

9810


In [75]:
predict_df.head()

,source,embedding,movie,0,ratings
0,9891,"[0.0020742759,0.0050524846,-0.003386466,0.0028...",4321,0,"[0,0,0,0,0,0,0,0,0,0]"
1,9891,"[0.0054219505,0.0052860463,-0.0050404333,-0.00...",135,0,"[0,0,0,0,0,0,0,0,0,0]"
2,9891,"[0.0024830783,-0.003575973,0.0061229025,0.0041...",5285,0,"[0,0,0,0,0,0,0,0,0,0]"
3,9891,"[0.0069850828,0.0008707234,0.0068662665,0.0029...",2741,0,"[0,0,0,0,0,0,0,0,0,0]"
4,9891,"[-0.006761877,0.0060815695,0.0057708365,0.0073...",559,0,"[0,0,0,0,0,0,0,0,0,0]"


In [76]:
predict_df = pd.read_csv('predict_test.csv')

Since we are using nodeIds here, I have to recreate the nodeIds to have the same id.

In [78]:
print(predict_df.head())
expected_df = predict_df[["ratings", "source", "movie"]]

predict_x = predict_df.loc[:, "embedding"]

   source                                          embedding  movie  0  \
0    9891  [0.0020742759,0.0050524846,-0.003386466,0.0028...   4321  0   
1    9891  [0.0054219505,0.0052860463,-0.0050404333,-0.00...    135  0   
2    9891  [0.0024830783,-0.003575973,0.0061229025,0.0041...   5285  0   
3    9891  [0.0069850828,0.0008707234,0.0068662665,0.0029...   2741  0   
4    9891  [-0.006761877,0.0060815695,0.0057708365,0.0073...    559  0   

                 ratings  
0  [0,0,0,0,0,0,0,0,0,0]  
1  [0,0,0,0,0,0,0,0,0,0]  
2  [0,0,0,0,0,0,0,0,0,0]  
3  [0,0,0,0,0,0,0,0,0,0]  
4  [0,0,0,0,0,0,0,0,0,0]  


In [79]:
predict_x = pd.DataFrame([sub.split(",") for sub in predict_x])
#predict_x = pd.DataFrame(np.array([np.array(item) for item in predict_x.values]))
predict_x.columns = [str(col) for col in predict_x.columns.get_values()]



C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,[0.0020742759,0.0050524846,-0.003386466,0.002862444,0.00557112,0.002237879,-0.001169441,-0.007466705,-0.0018846508,-0.0026723407,...,-0.00095538155,0.0060781054,-0.006508058,-0.0020968332,-0.00061050185,-0.00022153075,0.0031861581,-0.00094912347,0.0021690875,-0.0014343751]
1,[0.0054219505,0.0052860463,-0.0050404333,-0.0065103793,0.0029423,0.00093608623,-0.002902685,-0.0050046993,-0.006511066,0.005662674,...,0.0025054498,0.007454763,-0.0070858933,0.007696981,0.0029311026,0.0048488313,-0.004549181,0.003987452,0.0034448318,0.0024883177]
2,[0.0024830783,-0.003575973,0.0061229025,0.004178299,0.0019051132,0.00072764076,0.0008645881,0.0061498433,0.0002999682,0.005600294,...,0.005215622,-0.0069602537,0.0039596274,0.0060902224,0.007455875,0.0026260286,0.0017336925,-0.005638495,-0.003803534,0.0073580784]
3,[0.0069850828,0.0008707234,0.0068662665,0.0029306384,-0.0034020022,-0.0017614625,0.00026415923,-0.0042918283,0.0019480693,0.005610656,...,-0.0011373039,-0.006068216,0.0047586504,-0.002103879,0.002875452,-0.007034545,0.0077809896,0.0059217717,-0.0040861224,0.0075356713]
4,[-0.006761877,0.0060815695,0.0057708365,0.0073650675,0.00021214629,0.005643109,0.007186707,-0.0051961355,0.005609921,-0.00660391,...,0.0070847776,-0.0014172879,0.0039330935,-0.005759567,-0.0075020203,0.007301927,-0.0058451365,-0.0060837604,-0.005616533,-0.005398597]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,[-0.002698713,0.006218801,-0.0077919387,-0.006110709,0.003944708,0.005787083,0.0067991405,0.006844533,0.006096161,-0.0039038134,...,0.007222826,0.00007354563,-0.00086594233,-0.003642436,0.0014499583,0.0016078291,0.0049042753,-0.005262406,-0.004805527,0.0071064224]
596,[0.0010814407,-0.006123218,0.0014991675,-0.004086072,0.0047220406,0.00059033063,-0.00032910967,0.0014812927,0.0011673814,-0.00088125106,...,-0.0015354764,-0.00043749536,-0.005216136,-0.0053904965,-0.005120135,-0.00048819205,0.00024717982,-0.00023823157,-0.0011707927,0.002331914]
597,[-0.006656878,-0.007336678,-0.0033994315,-0.0025859498,0.0072017834,0.007331917,-0.002995128,-0.005485141,-0.003019573,-0.0065631494,...,0.0046523786,-0.007166286,-0.0015471169,-0.0049862307,-0.005731581,-0.006960357,0.0068321973,-0.004985357,-0.003724244,0.0071719056]
598,[-0.00018177432,0.004892791,0.0070315786,-0.0052663307,0.0006688544,0.004919152,0.007121826,-0.0059099006,-0.0016583222,0.0021765295,...,-0.0028237654,0.0075489604,-0.003454289,-0.0013383521,0.0048158346,-0.005009686,0.0031217514,0.00457758,0.003163943,0.001817258]


In [88]:
predict_x['0']= predict_x['0'].str.replace('[', '') 
predict_x['63']= predict_x['63'].str.replace(']', '') 
predict_x = predict_x.apply(pd.to_numeric)

In [104]:

def predict_input_fn(predict_x, num):
    return  tf.data.Dataset.from_tensors(dict(predict_x))

from tensorflow_core.estimator import inputs

predictions = classifier.predict(input_fn=lambda: predict_input_fn(predict_x, 100))


In [105]:
#user movie 
print('Predictions: {}'.format(str(predictions)))
columns=['User_ID', 'Movie', 'Ratings', 'Ratings_percent']
lst = []
for pred_dict, expec in zip(predictions, expected_df.as_matrix()):
    expected, source, movie = expec
    #print(source, movie, pred_dict)
    #print("Movie: {0}".format(movie))
    count=0
    for idx, label in enumerate(expected):
        if count <= 9:
            lst.append([source, movie, ratings[idx], pred_dict["probabilities"][idx]])
            count+=1
    #print("--")
    
df1 = pd.DataFrame(lst, columns=columns)

Predictions: <generator object Estimator.predict at 0x000002B29BEB7548>


C:\Users\meetn\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\meetn\AppData\Local\Temp\tmpclz8gzqr\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [107]:
#User and the movies that they ahve not rated. Prediction of each ratings per user
df1.head(20)

,User_ID,Movie,Ratings,Ratings_percent
0,9891,4321,0.5,0.015418
1,9891,4321,1.0,0.022571
2,9891,4321,1.5,0.016499
3,9891,4321,2.0,0.062595
4,9891,4321,2.5,0.047351
5,9891,4321,3.0,0.203710
6,9891,4321,3.5,0.112429
7,9891,4321,4.0,0.286891
8,9891,4321,4.5,0.097293
9,9891,4321,5.0,0.146711


In [109]:
df1.to_csv("predicted_out.csv")

In [110]:
#Sort the data frame based on Ratings_percent
df2 = df1.sort_values('Ratings_percent',ascending = False).groupby('User_ID').head(10)


In [111]:
df2.head(50)

,User_ID,Movie,Ratings,Ratings_percent
2127,5150,858,4.0,0.286970
847,9891,1620,4.0,0.286949
3017,9810,2276,4.0,0.286948
687,9891,2276,4.0,0.286948
5877,9889,228,4.0,0.286946
4217,9829,1542,4.0,0.286945
1087,9957,1542,4.0,0.286945
2597,5150,2137,4.0,0.286945
2057,5150,1983,4.0,0.286942
1017,9957,582,4.0,0.286942


In [113]:
df2.to_csv('Node2VecRatings_Output_Eval.csv')